# PNM: Physiological Noise Modelling

we used Pnm in FSL to find the voxel-wise regresorse for respiration signal to model in GLM
<br>
https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/PNM/UserGuide
<br>
https://www.jiscmail.ac.uk/cgi-bin/webadmin?A2=FSL;f099cf08.1605


-----------------------------------------------------------
Script written by Mehdi Behroozi
<br>
Biopsychology, 
<br>
Ruhr-University Bochum, Bochum, Germany
<br>
(2022.03)

-----------------------------------------------------------

In [ ]:
import os
import glob
data_path = '/mnt/d/Data/Pigeon/Sleep/analysis3'
sub_dirs = glob.glob('%s/sub*/2*/results'%(data_path))

for curr_sub in sub_dirs:
    print(curr_sub)
    os.system('fslFixText %s/PNM/Physiological_record.txt %s/PNM/mypnm_input.txt'%(curr_sub,curr_sub))
    os.system('pnm_stage1 -i %s/PNM/mypnm_input.txt -o %s/PNM/mypnm -s 100 --tr=4 --smoothcard=0.1 --smoothresp=0.1 --resp=2 --cardiac=3 --trigger=1 --rvt'%(curr_sub,curr_sub))
    os.system('popp -i %s/PNM/mypnm_input.txt -o %s/PNM/mypnm -s 100 --tr=4 --smoothcard=0.1 --smoothresp=0.1 --resp=2 --cardiac=3 --trigger=1 --rvt'%(curr_sub,curr_sub))
    os.system('pnm_evs -i %s/preprocessed_data_bandpassed_chopped.nii.gz -c %s/PNM/mypnm_card.txt -r %s/PNM/mypnm_resp.txt -o %s/PNM/mypnm --tr=4 --oc=0 --or=4 --multc=0 --multr=2 --rvt=%s/PNM/mypnm_rvt.txt --rvtsmooth=10 --sliceorder=interleaved_up --slicedir=z'%(curr_sub,curr_sub,curr_sub,curr_sub,curr_sub))
    os.system('ls %s/PNM/mypnmev* >> %s/PNM/mypnm_evlist.txt'%(curr_sub,curr_sub))


# for resting-state data
If you are doing custom preprocessing and want to regress out the voxelwise confounds from PNM, then it should be possible to do this with fsl_glm and the voxelwise confound option, something like:

fsl_glm -i input --vxf=voxelwiseConfound1,voxelwiseConfound2,etc --out_res=output -d blank.mat

should work - the "residuals" output being the original data corrected for the confounds. blank.mat should be a single column of zeros, with a number of rows equal to the number of timepoints ( you can create this with Glm_gui ).

you can use evlist.txt to find the names of the files to be passed to the vxf option.

Note that if input has already been filtered ( e.g. highpass ) the same filtering must be applied to the voxelwise EV's.